<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python**

In [4]:
%%writefile tp2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validateArgs(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(min=MIN_MATRIX_SIZE, max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrixGenerator(size):
    matrixResult=[]
    for i in range(0, size):
        matrixRow=[]
        for j in range(0, size):
            matrixRow.append(random.randint(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrixResult.append(matrixRow)
    return matrixResult

def printMatrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuentialMult(size, matrix_a, matrix_b):
    matrixResult = []
    for i in range(0, size):
        rowResult = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                rowResult[j] = matrix_a[i][k] * matrix_b[k][j]
        matrixResult.append(rowResult)
    return matrixResult

def multiplyRow(size, matrix_a, matrix_b, rowResult, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            rowResult[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

def concurrentMult(size, matrix_a, matrix_b):
    matrixResult = []
    threads = []

    for i in range(0, size):
        rowResult = [None] * size
        thread = Thread(target=multiplyRow, args=(size, matrix_a, matrix_b, rowResult, i))
        thread.start()
        threads.append(thread)
        matrixResult.append(rowResult)

    for thread in threads:
        thread.join()

    return matrixResult

def compareMatrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validateArgs(sys.argv)
    matrix_a = matrixGenerator(size)
    matrix_b = matrixGenerator(size)
    matrix_secuential = secuentialMult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrentMult(size,matrix_a,matrix_b)

    printMatrix("A", matrix_a)
    printMatrix("B", matrix_b)
    printMatrix("Secuencial", matrix_secuential)
    printMatrix("Concurrente", matrix_concurrent)

    resultCompare = "iguales" if compareMatrix(size, matrix_secuential, matrix_concurrent) else "distintas"
    print("Las matrices concurrente y secuencial son {compare}".format(compare=resultCompare))

main()

Overwriting tp2.py


In [5]:
!python3 tp2.py 1

El tamaño debe estar entre 5 y 20


In [6]:
!python3 tp2.py 7


Matrix: A
	-13		23		-4		5		-1		14		25	
	28		-20		13		-28		30		13		-11	
	-20		-15		-23		0		-1		-2		30	
	14		-3		-1		-17		7		28		32	
	22		-16		-2		-32		12		-3		-29	
	-5		10		-26		4		28		27		13	
	23		-24		17		25		28		20		28	

Matrix: B
	10		-21		-18		-29		-8		24		-11	
	-17		-23		-18		-23		13		2		-25	
	17		18		-30		14		-16		16		-6	
	-6		5		28		-10		-8		10		-25	
	15		26		15		12		-5		-17		3	
	5		-23		17		-12		12		26		7	
	32		28		-17		-19		-17		7		32	

Matrix: Secuencial
	800		700		-425		-475		-425		175		800	
	-352		-308		187		209		187		-77		-352	
	960		840		-510		-570		-510		210		960	
	1024		896		-544		-608		-544		224		1024	
	-928		-812		493		551		493		-203		-928	
	416		364		-221		-247		-221		91		416	
	896		784		-476		-532		-476		196		896	

Matrix: Concurrente
	800		700		-425		-475		-425		175		800	
	-352		-308		187		209		187		-77		-352	
	960		840		-510		-570		-510		210		960	
	1024		896		-544		-608		-544		224		1024	
	-928		-812		493		551		493		-203		-928	
	416		364		-221		-247		-221		91		416